In [9]:
from ultralytics import YOLO
from collections import Counter
import cv2

In [10]:
model = YOLO("../runs/yolov11_fruit_veg/weights/best.pt")

In [11]:
def run_detection(image, conf_thresh=0.4):
    results = model(image, conf=conf_thresh)[0]
    
    detections = []
    for box in results.boxes:
        detections.append({
            "class_id": int(box.cls),
            "class_name": results.names[int(box.cls)],
            "confidence": float(box.conf),
            "bbox": box.xyxy.tolist()[0]
        })
    return detections

In [12]:
img = cv2.imread("../assets/example_images/sample.jpg")
detections = run_detection(img)

detections


0: 640x640 20 0s, 9.0ms
Speed: 7.2ms preprocess, 9.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


[{'class_id': 0,
  'class_name': '0',
  'confidence': 0.8126479983329773,
  'bbox': [340.82305908203125,
   1020.1770629882812,
   753.2926635742188,
   1383.989501953125]},
 {'class_id': 0,
  'class_name': '0',
  'confidence': 0.7927098870277405,
  'bbox': [358.40380859375,
   734.3424682617188,
   631.9544677734375,
   1014.3772583007812]},
 {'class_id': 0,
  'class_name': '0',
  'confidence': 0.762939453125,
  'bbox': [1044.4283447265625,
   386.841552734375,
   1417.8846435546875,
   645.7745971679688]},
 {'class_id': 0,
  'class_name': '0',
  'confidence': 0.7353485226631165,
  'bbox': [147.09141540527344,
   752.1712646484375,
   415.0992126464844,
   1006.635009765625]},
 {'class_id': 0,
  'class_name': '0',
  'confidence': 0.7294889092445374,
  'bbox': [619.8507080078125,
   172.7032012939453,
   963.277099609375,
   367.2476501464844]},
 {'class_id': 0,
  'class_name': '0',
  'confidence': 0.7112269401550293,
  'bbox': [141.799560546875,
   108.2962646484375,
   325.2948608398

In [13]:
def count_items(detections, conf_thresh=0.4):
    filtered = [
        d["class_name"] for d in detections
        if d["confidence"] >= conf_thresh
    ]
    return dict(Counter(filtered))

In [14]:
count_items(detections)

{'0': 20}